In [36]:
import pandas as pd
pd.set_option('display.max_rows', None)
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from decimal import Decimal

In [37]:
# Read in the train_rev1 datafile downloaded from kaggle
df = pd.read_csv('new_combined_results.csv')
df.head(2)

,id,title,company,description,source,location,min_salary,max_salary
0,1,Senior Software Engineer - Remote,Akamai,**Are you a software developer who loves solvi...,indeed.com,United States,"$113,430","$170,043"
1,2,Software Engineer II,Microsoft,Microsoft’s Azure Data engineering team is lea...,indeed.com,United States,"$94,300","$182,600"


In [38]:
# Function to convert salary strings to Decimal and add the corresponding label
def convert_salary(salary_str):
    if isinstance(salary_str, str):
        # Use regular expressions to extract the numeric part of the string
        match = re.search(r'\d+(\.\d+)?', salary_str)
        if match:
            numeric_part = match.group()
            numeric_value = Decimal(numeric_part.replace(',', ''))  # Convert to Decimal
            if 'K' in salary_str:
                return int(numeric_value), 'per year'
            elif ',' in salary_str:
                return int(numeric_value), 'per year'
            elif '.' in salary_str:
                return int(numeric_value), 'per hour'
            else:
                return int(numeric_value), 'per hour'
    else:
        # If it's already a number (no conversion needed)
        return salary_str, ''


# Apply the function to 'min_salary' and 'max_salary' columns
df['min_salary_year'], df['salary_type'] = zip(*df['min_salary'].apply(convert_salary))
df['max_salary_year'], _ = zip(*df['max_salary'].apply(convert_salary))

# Calculate annual salary based on 40 hours/week and 52 weeks/year for rows with 'on_hours'
df.loc[df['salary_type'] == 'per hour', ['min_salary_year', 'max_salary_year']] *= 40 * 52
df.loc[df['salary_type'] == 'per hour', ['min_salary_year', 'max_salary_year']] /= 1000

df = df.fillna(value=0)
df = df.astype({"min_salary_year":'int', "max_salary_year":'int'})
# df.to_csv('your_file.csv', index=False) 

In [39]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yuliiaantonova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
def convert_utf8(s):
    return str(s)

df['description'] = df['description'].map(convert_utf8)

In [41]:
# Define a function to extract experience information
def extract_experience(text):
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    # Define a pattern to match "2 years," "2+ years," "3 years," "5 years," etc.
    experience_pattern = r'(\d+)(\s*[\+]?|[\+]\s*)years'

    # Extract years of experience from the tokenized text
    years_of_experience = None
    for sentence_words in words:
        sentence = ' '.join(sentence_words)
        match = re.search(experience_pattern, sentence, re.IGNORECASE)
        if match:
            years_of_experience = match.group(0)
            break

    return years_of_experience

# Apply the function to the "description" column and create a new column "experience"
df['experience'] = df['description'].apply(extract_experience)

# Display the modified DataFrame
# print(df['experience'])

In [42]:
# Define a function to extract experience information
def extract_experience(text):
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    # Define a pattern to match "2 years," "2+ years," "3 years," "5 years," etc.
    experience_pattern = r'(\d+)(\s*[\+]?|[\+]\s*)years'

    # Extract years of experience from the tokenized text
    years_of_experience = None
    for sentence_words in words:
        sentence = ' '.join(sentence_words)
        match = re.search(experience_pattern, sentence, re.IGNORECASE)
        if match:
            years_of_experience = int(match.group(1))
            break

    return years_of_experience

# Apply the function to the "description" column and create a new column "experience"
df['experience'] = df['description'].apply(extract_experience)

# Define a function to categorize experience levels
def categorize_experience(row):
    title = row['title']
    experience = row['experience']

    if not pd.isna(experience):
        if 'software' in title.lower():
            if experience <= 2:
                return "Junior Software Engineer"
            elif 3 <= experience <= 4:
                return "Middle Software Engineer"
            else:
                return "Senior Software Engineer"
        elif 'data' in title.lower():
            if experience <= 2:
                return "Junior Data Engineer"
            elif 3 <= experience <= 4:
                return "Middle Data Engineer"
            else:
                return "Senior Data Engineer"
    return "Other"

# Apply the function to categorize experience levels and create a new column "experience_level"
df['experience_level'] = df.apply(categorize_experience, axis=1)

In [43]:
grouped_df = df.groupby('experience_level')

# Display the groups (optional)
# for group, group_data in grouped_df:
#     print(f"Group: {group}")
#     print(group_data)

# If you want to concatenate the groups into a new DataFrame, you can use the following:
# Concatenate all groups into a new DataFrame
new_df = pd.concat([group_data for _, group_data in grouped_df], ignore_index=True)

# Display the new DataFrame

In [27]:
df.columns.values

array(['id', 'title', 'company', 'description', 'source', 'location',
       'min_salary', 'max_salary', 'min_salary_year', 'salary_type',
       'max_salary_year', 'experience', 'experience_level'], dtype=object)

In [47]:
# Assuming 'df' is your DataFrame
result_df = df.groupby(['experience_level'])[['min_salary_year', 'max_salary_year']].mean().reset_index()
result_df.columns = ['experience_level', 'avg_min_salary', 'avg_max_salary']
result_df['avg_min_salary'] = result_df['avg_min_salary'].round(0).astype(int)
result_df['avg_max_salary'] = result_df['avg_max_salary'].round(0).astype(int)
result_df

,experience_level,avg_min_salary,avg_max_salary
0,Junior Data Engineer,80,105
1,Junior Software Engineer,91,129
2,Middle Data Engineer,98,125
3,Middle Software Engineer,103,151
4,Other,90,122
5,Senior Data Engineer,104,130
6,Senior Software Engineer,107,149


In [35]:
# a = df.to_dict('records')